# **Scrapping Data User Review**

**Install dan Import Library**

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Pencarian FIlm**

In [3]:
#Search Keyword
keySearch = "Ivanna";

In [4]:
# URL API
url = 'https://imdb-api.com/en/API/Search/k_a74bxqbp/'+keySearch

# request data dari API
response = requests.get(url)

# memeriksa apakah request berhasil
if response.status_code != 200:
    raise ValueError('Request tidak berhasil, status code:', response.status_code)

# mengambil data dari response JSON
json_data = response.json()

# mengubah data JSON menjadi dataframe
df = pd.DataFrame(json_data['results'])

# menampilkan dataframe
df.head()

,id,resultType,image,title,description
0,tt11448138,Title,https://m.media-amazon.com/images/M/MV5BYzc3Nj...,Ivanna,"2022 Caitlin Halderman, Jovarel Callum"
1,tt14651512,Title,https://m.media-amazon.com/images/M/MV5BMzJhMz...,Life of Ivanna,"2021 Ivanna Yaptune, Yaroslav Yaptune"
2,tt0065905,Title,https://m.media-amazon.com/images/M/MV5BZWJiZG...,Scream of the Demon Lover,"1970 Erna Schurer, Carlos Quiney"
3,tt7178984,Title,https://m.media-amazon.com/images/M/MV5BMTc0Nj...,The Dance,"2019 Irene Esser, Gabriel Agüero"
4,tt0174784,Title,https://m.media-amazon.com/images/M/MV5BZWMzYz...,Ivanna,"1959 Inna Burduchenko, Anatoli Motornyj"


In [5]:
#Select Film dari pencarian 
#Isi menurut index
idnum = 0;

# Mengambil data dari baris pertama dan kolom kedua
idFilm = df.iloc[idnum, 0]
namaFilm = df.iloc[idnum, 3]

print( idFilm+" "+ df.iloc[idnum, 3]+ " " + df.iloc[idnum, 4])

tt11448138 Ivanna 2022 Caitlin Halderman, Jovarel Callum


**Scrapping Data**

In [6]:
url = "https://www.imdb.com/title/" + idFilm + "/reviews"
response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

reviews = soup.find_all("div", {"class": "lister-item-content"})

data = {"Title": [], "Content": [], "Rating": []}

for review in reviews:
    title = review.find("a").text.strip()
    content = review.find("div", {"class": "text show-more__control"}).text.strip()
    rating_elem = review.find("div", {"class": "ipl-ratings-bar"})
    rating = rating_elem.find("span", {"class": "rating-other-user-rating"}).find("span").text.strip() if rating_elem else ""
    data["Title"].append(title)
    data["Content"].append(content)
    data["Rating"].append(rating)

load_more_link = soup.find("div", {"class": "load-more-data"})
if load_more_link is not None:
    load_more_link = load_more_link.get("data-key")
    while load_more_link:
        url = f"https://www.imdb.com/title/{idFilm}/reviews/_ajax?paginationKey={load_more_link}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        reviews = soup.find_all("div", {"class": "lister-item-content"})
        for review in reviews:
            title = review.find("a").text.strip()
            content = review.find("div", {"class": "text show-more__control"}).text.strip()
            rating_elem = review.find("div", {"class": "ipl-ratings-bar"})
            rating = rating_elem.find("span", {"class": "rating-other-user-rating"}).find("span").text.strip() if rating_elem else ""
            data["Title"].append(title)
            data["Content"].append(content)
            data["Rating"].append(rating)
        load_more_link = soup.find("div", {"class": "load-more-data"})
        if load_more_link is not None:
            load_more_link = load_more_link.get("data-key")
else:
    for review in reviews:
        title = review.find("a").text.strip()
        content = review.find("div", {"class": "text show-more__control"}).text.strip()
        rating_elem = review.find("div", {"class": "ipl-ratings-bar"})
        rating = rating_elem.find("span", {"class": "rating-other-user-rating"}).find("span").text.strip() if rating_elem else ""
        data["Title"].append(title)
        data["Content"].append(content)
        data["Rating"].append(rating)

reviewdf = pd.DataFrame(data)

print(len(reviewdf))
reviewdf.head()


18


,Title,Content,Rating
0,This is a Very Good Horror Movie better than D...,The Origin Story of Ivanna before Danur 2.Good...,8
1,BLOODY SLASHER!,Good bloody slasher. Camerawork on point! Ther...,7
2,Another awful entry in an all-around awful series,Kimo Stamboel's direction brings a touch of gr...,4
3,"Pretty okay, though somewhat repetitive",The plot works as it is. It's simple but the m...,6
4,It's just another ordinary horror movie.,The storyline is pretty simple and predictable...,4


In [7]:
#Menggabungkan dua kolom menjadi satu kolom yang bernama komentar
reviewdf['UserReview'] = reviewdf.apply(lambda row: row['Title'] + ' ' + row['Content'], axis=1)
reviewdf.drop(columns = ['Title', 'Content'], inplace = True)
reviewdf.columns = ['Rating','UserReview']
reviewdf.head()

,Rating,UserReview
0,8,This is a Very Good Horror Movie better than D...
1,7,BLOODY SLASHER! Good bloody slasher. Camerawor...
2,4,Another awful entry in an all-around awful ser...
3,6,"Pretty okay, though somewhat repetitive The pl..."
4,4,It's just another ordinary horror movie. The s...


In [8]:
reviewdf.to_csv('/content/drive/MyDrive/Colab Notebooks/dataset/Scrapping_'+namaFilm+'.csv', index=False)